# Rota mais valiosa
Instância Um grafo completo G = (V, A) com distâncias da, a ∈ A, entre os vértices, prêmios pv,
v ∈ V , nos vértices e um vértice de origem vo ∈ V .
Solução Uma rota R = (vo, v1, v2, . . . , vk, vo) inciando e terminando em vo com vértices intermediários vi ∈ V . Cada vértice em V \ {vo} pode ser contido no máximo um vez em R.
Objetivo Minimizar o valor total v(R) = dvo,v1 +
∑
i∈[k−1] dvi,vi+1 + dvk,vo −
∑
i∈[k]
pvi da rota.

## Variáveis
- D -> Matriz de adjacência com o peso de cada aresta 
- A -> Matriz de adjacência com a booleano informando se a aresta pertence ou não a solução
- P -> Lista com o peso (prêmio) de cada vértice
- V -> Lista com a booleano informando se o vértice foi visitado ou não

## Modelagem
Minimizar 
    $\sum_{i=0}^{V-1}\sum_{j=0}^{V-1} D_{(i,j)} * A_{(i,j)} - \sum_{i=0}^{V-1} P_{(i)} * V_{(i)}$


Sujeito:
    
 - A aresta i,j pode (1) ou não (0) estar contido na solução: 
     
     $A_{(i,j)} \in \{0,1\}$ 
 
 
 - O vértica i pode (1) ou não (0) estar contido em algumas das arestas da solução:
     
     $V_{(i)} \in \{0,1\}$ 
 
 
 - A aresta que sai do vértice 0, que é vértice de partida, deve obrigatoriamente estar contido na solução
     
     $\sum_{j=1}^{V} A_{(0,j)} = 1$
 
 
 - A aresta que chega no vértice 0, que é vértice de destino, deve obrigatoriamente estar contido na solução
     
     $\sum_{i=1}^{V} A_{(i,1)} = 1$
 
 
 - Vinculação dos vértice visitados com os prêmios destes.
     
     $\sum_{j=1}^{V} A_{(i,j)} = V_{i},\ \forall_{i} \in \{1,..,V\}$
 
 
 - Um vértice não pode estar contido como destino em mais de 1 aresta da solução
     
     $\sum_{i=1}^{V-1} A_{(i,j)} <= 1,\ \forall_{j} \in \{1,..,V\}$
 
 
 - Um vértice não pode estar contido como partida em mais de 1 aresta da solução
     
     $\sum_{j=1}^{V} A_{(i,j)} <= 1,\ \forall_{i} \in \{1,..,V\}$
     
     
 - Um aresta não pode conter o mesmo vértice como partida e destino
     
     $Diagonal(A) <> Matriz\ identidade$
 
 
 - Se um vértice está contido como partida em alguma aresta da solução, ele obrigatoriamente deve estar contido como destino em alguma aresta da solução 
     
     $\sum_{i=1}^{V} A_{(i,j)} = \sum_{i=1}^{V} A_{(j, i)}\ \forall_{j} \in \{1,..,V\} $

## Código

In [1]:
using NBInclude
using GLPK
using JuMP
@nbinclude("parser.ipynb")

In [2]:
function get_diagonal(matrix)
    diagonal = []
    cardinalidade = size(matrix)[1]
    for i in 1:cardinalidade
        push!(diagonal, matrix[i, i])
    end
    return diagonal
end

get_diagonal (generic function with 1 method)

In [22]:
file = "instancias/A-n32-k5.vrp"
v, P = parse_file(file)
D = get_distancias(v)
cardinalidade = length(P);

In [23]:
# m = Model(GLPK.Optimizer)

# @variable(m, A[1:5, 1:5], Bin)
# @variable(m, V[1:5], Bin);

# @objective(m, Min, sum(D[1:5, 1:5] .* A[:5,:5]) - sum(P[1:5] .* V[1:5]));

In [24]:
m = Model(GLPK.Optimizer)

@variable(m, A[1:cardinalidade, 1:cardinalidade], Bin)
@variable(m, V[1:cardinalidade], Bin);

@objective(m, Min, sum(D .* A) - sum(P .* V));

In [26]:
@constraints(m, begin
    sum(A[1, :])               == 1
    sum(A[:, 1])               == 1
    transpose(sum(A, dims=1)) .== V
    sum(A, dims=2)            .<= ones(cardinalidade)
    sum(A, dims=1)            .<= ones((1, cardinalidade))
    get_diagonal(A)           .== zeros(cardinalidade)
    transpose(sum(A, dims=1)) .- sum(A, dims=2) .== zeros(cardinalidade)
    end
)

In [27]:
optimize!(m)

In [28]:
@show objective_value(m)

objective_value(m) = -133.0


-133.0

In [31]:
maximum(sum(JuMP.value.(A), dims=2) .== transpose(sum(JuMP.value.(A), dims=1)))

true

In [32]:
for i in 1:32
    println(i," ", argmax(JuMP.value.(A)[i,:]))
end

1 31
2 13
3 4
4 3
5 12
6 1
7 1
8 14
9 1
10 23
11 1
12 5
13 17
14 8
15 1
16 1
17 2
18 20
19 1
20 18
21 1
22 32
23 10
24 1
25 28
26 1
27 1
28 25
29 1
30 1
31 1
32 22


In [ ]:
JuMP.value.(A)[7,1]

In [ ]:
@show value(P[1])

In [ ]:
@show value(P[31])

In [ ]:
@show value(P[27])

In [ ]:
@show value(D[1, 27])

In [ ]:
@show value(D[27, 31])

In [ ]:
@show value(D[1, 31])